The general optimization problem is \\
$$\text{min} \ \  \sum_{i=1}^{6} (P_i-\bar{P_i})^2 \\
\text{s.t.  } \\
\bar{P_i}=b_0+b_1L_i+b_2E_i \ \ \forall i  \\
b_0 \geq 0$$ \\


In [ ]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

In [ ]:
data_csvfile = pd.read_csv('lab6_ex3.csv')

In [ ]:
data_csvfile.columns

Index(['warehouse', 'seling price', 'lot size', 'elevation'], dtype='object')

In [ ]:
# create a model
model = ConcreteModel()

In [ ]:
# here N denotes no. of variables
N = len(data_csvfile.index)
n = 3 

In [ ]:
col_indices = np.arange(N)
b_indices = np.arange(n)

In [ ]:
#declaring the decision variables in the model
model.P_ = Var(col_indices)   #P bar is denoted by P_
model.b = Var(b_indices)

In [ ]:
#objection function
model.objective = Objective(expr=sum((data_csvfile['seling price'][i]-model.P_[i])**2 for i in col_indices),sense=minimize)

In [ ]:
model.constraints = ConstraintList()

In [ ]:
for i in col_indices:
  model.constraints.add(model.b[0]+model.b[1]*data_csvfile['lot size'][i]+model.b[2]*data_csvfile['elevation'][i]- model.P_[i] == 0)

In [ ]:
model.b[0].setlb(0)

In [ ]:
model.pprint()

3 Set Declarations
    P__index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

2 Var Declarations
    P_ : Size=6, Index=P__index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 :  None :  None :  None : False :  True :  Reals
          5 :  None :  None :  None : False :  True :  Reals
    b : Size=3, Index=b_index
       

In [ ]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print(u'P\u0304[',i,']=', model.P_[i].value)  
for i in b_indices:
  print('b[',i,']=',model.b[i].value)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 9
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.023651838302612305
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  4291134028.8074546

Decision Variables
P̄[ 0 ]= 126237.55560262657
P̄[ 1 ]= 135941.27303537386
P̄[ 2 ]= 153366.60665113322
P̄[ 3 ]= 116533.83816987927
P̄[ 4 ]= 93331.39165431052
P̄[ 5 ]= 114589.33488667656
b[ 0 ]= 104904.41643719551
b[ 1 ]= 38.70207583139166
b[ 2 ]= -193.32238932429576


In [ ]:
k=0
for i in col_indices:
  k=k+abs(data_csvfile['seling price'][i]-model.P_[i].value)
k

136463.67295064603

In [ ]:
print('For this model: ')
for i in col_indices:
  print('R[',i+1,']= ',abs(data_csvfile['seling price'][i]-model.P_[i].value))

For this model: 
R[ 1 ]=  48762.44439737343
R[ 2 ]=  14058.726964626141
R[ 3 ]=  28366.606651133217
R[ 4 ]=  26533.838169879265
R[ 5 ]=  13331.391654310515
R[ 6 ]=  5410.665113323441


So, maximum deviation in this model is 48762.4444 .

From Linear model with absolute terms.

Optimal sum of residuals =  125454.55

R[ 1 ]= 38818.182 \\
R[ 2 ]= -2.7009345e-11 \\
R[ 3 ]= 54272.727 \\
R[ 4 ]= 32363.636 \\
R[ 5 ]= -2.7785522e-11 \\
R[ 6 ]= -1.671118e-11 \\

So, for Linear model with absolute terms, the maximum deviation is 54272.727 .

The maximimum deviation is lower in non linear program because in that we are minimizing the sum of squares.Since, increase in deviation increases the objective quadraticaly. In this if we have more deviation, then due to square the objective value could not be made minimum. Or simply because $x^2+y^2 >= ((x+y)/2)^2$. Hence the optimization tried to keep deviation for all i evenly distributed resulting decrease in max deviation